## **5. SUPPORT VECTOR MACHINE (LinearSVC)**

In [ ]:
# Import necessary libraries
import os
import joblib
import pandas as pd
from sklearn.svm import LinearSVC
from scipy.stats import loguniform
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# List drive content
os.listdir('/content/drive/MyDrive')

['LC_loans_granting_model_dataset.csv',
 'Colab Notebooks',
 'processed_loan_data.csv',
 'processed_loan_data.parquet',
 'LCDataDictionary.xlsx',
 'NOTE2.ipynb',
 'loan_2019_2020.csv',
 'loan_2019_2020_filtered.csv',
 'loan_2019_2020_final.csv',
 'Portfolio - Bertie Dickinson.ipynb',
 'd1_cleaned.csv',
 'categorical_variable_plots.png',
 'categorical_default_non_default_rate_plots.png',
 'Default_Rate_Of_Categorical_variable.png',
 'numerical_density_plots_by_default.png',
 'Categorical_Default_non_Default_Count.png',
 'Numerical_density_plots_by_default.png',
 'FE_data.csv',
 'Feature_Engineering_data.csv',
 'processed_dataset.xls',
 'Categorical_Default_non_Default_Rate.png',
 'PHASE_2_data.csv',
 'project_data',
 'Phase_2_data_splits',
 'Phase_2_data_preprocessed',
 'Phase_2_data_preprocessed_csv',
 'Phase_2_models',
 'sd_30.csv',
 'P1_data.csv',
 'P2_data.csv',
 'split_P2_data',
 'First.ipynb',
 'Phase_3_models',
 'Innitial Data Cleaning.ipynb',
 'sd_50.csv',
 'Copy of Copy of PHAS

In [ ]:
# Path to your saved file
joblib_filename = "/content/drive/MyDrive/Phase_2_data_preprocessed/preprocessed_data.pkl"

# Load the dictionary
preprocessed_data = joblib.load(joblib_filename)

# Unpack the dictionary
X_train_df = preprocessed_data['X_train_df']
y_train = preprocessed_data['y_train']
X_val_df = preprocessed_data['X_val_df']
y_val = preprocessed_data['y_val']
X_test_df = preprocessed_data['X_test_df']
y_test = preprocessed_data['y_test']

print("✅ Preprocessed data loaded successfully.")

✅ Preprocessed data loaded successfully.


In [ ]:
# Path to your saved file
joblib_filename = "/content/drive/MyDrive/Phase_2_data_preprocessed/preprocessed_data.pkl"

# Load the dictionary
preprocessed_data = joblib.load(joblib_filename)

# Unpack the dictionary
X_train_df = preprocessed_data['X_train_df']
y_train = preprocessed_data['y_train']
X_val_df = preprocessed_data['X_val_df']
y_val = preprocessed_data['y_val']
X_test_df = preprocessed_data['X_test_df']
y_test = preprocessed_data['y_test']

print("✅ Preprocessed data loaded successfully.")

# Print the shapes to verify
print("X_train_df shape (for tuning/training):", X_train_df.shape)
print("y_train shape (for tuning/training):", y_train.shape)
print("X_val_df shape (for early stopping/validation):", X_val_df.shape)
print("y_val shape (for early stopping/validation):", y_val.shape)
print("X_test shape:", X_test_df.shape)
print("y_test shape:", y_test.shape)

✅ Preprocessed data loaded successfully.
X_train_df shape (for tuning/training): (301926, 78)
y_train shape (for tuning/training): (301926,)
X_val_df shape (for early stopping/validation): (43100, 78)
y_val shape (for early stopping/validation): (43100,)
X_test shape: (86243, 78)
y_test shape: (86243,)


In [ ]:
# ---------------------------
# 1. Define model and pipeline
# ---------------------------
# LinearSVC is sensitive to feature scaling, so we use a pipeline with StandardScaler
svm_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', LinearSVC(
        dual=True, # Recommended for n_samples > n_features, or when n_features is similar to n_samples
        random_state=42,
        class_weight='balanced', # Handle class imbalance
        max_iter=1000 # Increased max_iter for convergence, will tune further
    ))
])

# ---------------------------
# 2. Define parameter grid for Randomized Search
# ---------------------------
param_distributions_svm = {
    'svc__C': loguniform(1e-5, 100), # Regularization parameter, log-uniform scale is often good
    'svc__penalty': ['l1', 'l2'],
    'svc__loss': ['hinge', 'squared_hinge'],
    'svc__max_iter': [1000, 2000, 3000, 5000] # Increased range for max_iter
}

print("✅ LinearSVC pipeline and parameter distributions defined.")

✅ LinearSVC pipeline and parameter distributions defined.


In [ ]:
# ---------------------------
# 3. Randomized Search with 3-fold CV, optimize ROC-AUC
# ---------------------------
random_search_svm = RandomizedSearchCV(
    estimator=svm_pipeline,
    param_distributions=param_distributions_svm,
    n_iter=20,  # Number of parameter settings to sample. Increase for more thorough search.
    scoring='roc_auc',
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
    n_jobs=-1,  # Use all available cores
    verbose=3,
    random_state=42,
    return_train_score=True
)

print("Fitting Randomized Search for LinearSVC...")
random_search_svm.fit(X_train_df, y_train)


Fitting Randomized Search for LinearSVC...
Fitting 3 folds for each of 20 candidates, totalling 60 fits


/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
27 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
21 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **la

RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
                   estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                             ('svc',
                                              LinearSVC(class_weight='balanced',
                                                        dual=True,
                                                        random_state=42))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'svc__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7836ded97a10>,
                                        'svc__loss': ['hinge', 'squared_hinge'],
                                        'svc__max_iter': [1000, 2000, 3000,
                                                          5000],
                                        'svc__penalty': ['l1', 'l2']},
                   random_state=42, return_train_score=True, scoring='roc_auc',
                   verbose=3)

In [ ]:
# ---------------------------
# 4. Display best results
# ---------------------------
print("\n✅ Best Parameters from Randomized Search:", random_search_svm.best_params_)
print("✅ Best ROC AUC Score (CV) from Randomized Search:", random_search_svm.best_score_)

# Save CV results
cv_results_df_svm = pd.DataFrame(random_search_svm.cv_results_)
display(cv_results_df_svm[['params', 'mean_test_score', 'std_test_score', 'mean_train_score']].sort_values(by='mean_test_score', ascending=False))


✅ Best Parameters from Randomized Search: {'svc__C': np.float64(0.43416618003617374), 'svc__loss': 'squared_hinge', 'svc__max_iter': 2000, 'svc__penalty': 'l2'}
✅ Best ROC AUC Score (CV) from Randomized Search: 0.7201868876130987


,params,mean_test_score,std_test_score,mean_train_score
18,"{'svc__C': 0.43416618003617374, 'svc__loss': '...",0.720187,0.000412,0.720837
8,"{'svc__C': 0.19185373703841915, 'svc__loss': '...",0.720186,0.000413,0.720837
15,"{'svc__C': 0.0004132917911457901, 'svc__loss':...",0.720185,0.000417,0.720833
6,"{'svc__C': 0.00019223460470643635, 'svc__loss'...",0.720181,0.000424,0.720825
1,"{'svc__C': 2.8697033315125156, 'svc__loss': 'h...",0.719968,0.000539,0.720578
17,"{'svc__C': 0.0001635131083842518, 'svc__loss':...",0.719909,0.000593,0.720525
2,"{'svc__C': 0.0001235838277230692, 'svc__loss':...",0.719809,0.000623,0.720422
4,"{'svc__C': 1.3934502251337587e-05, 'svc__loss'...",0.719615,0.000582,0.720204
13,"{'svc__C': 1.2383822772948166e-05, 'svc__loss'...",0.715866,0.000874,0.716281
5,"{'svc__C': 37.14231450644701, 'svc__loss': 'sq...",0.667367,0.000797,0.667590


In [ ]:
# ---------------------------
# 5. Save the best model
# ---------------------------
best_svm_model = random_search_svm.best_estimator_

save_path = "/content/drive/MyDrive/Phase_2_models/" # Ensure this path exists or create it
os.makedirs(save_path, exist_ok=True)
svm_model_filename = os.path.join(save_path, 'best_linear_svc_auc.pkl')

joblib.dump(best_svm_model, svm_model_filename)
print(f"\n✅ Best LinearSVC model saved to: {svm_model_filename}")


✅ Best LinearSVC model saved to: /content/drive/MyDrive/Phase_2_models/best_linear_svc_auc.pkl


_